# Docker Shortcuts

## About Docker

Docker is an open platform for developing, shipping and running applications. GridAPPS-D uses Docker to package applications, services, and the individual components of the GridAPPS-D Platform.

Each application and underlying platform component run in a loosely isolated environment known as a Docker Container. Each container is assigned a unique identifier string and a human-readable name which are used to interact with that container during runtime.

More detailed information about Docker and Docker Containers is available on [Docker Docs](https://docs.docker.com/get-started/overview/)

---

## Managing Running Containers

### View Running Containers

To view all containers that are currently running, use the `docker ps` command.

A list of all containers with the container ID, image name, and ports currently used will be printed to the terminal window. 

![docker-ps](images/1.6/docker_ps.png)

---

### Enter a Running Container

To enter a container, use the `docker exec` command with the container name. For example, to enter the GridAPPS-D container, run

`docker exec -it gridappsd /bin/bash`

Note that earlier releases of the GridAPPS-D platform did not use consistent names, so if you are running a 2021 or earlier release, it is necessary to use the container ID obtained from the `docker ps` command:

`docker exec -it 123con45name /bin/bash`


![docker-exec](images/1.6/docker_exec.png)

---

### View Contents of a Container

After entering a container with `docker exec -it containername /bin/bash`, use the `ls -l` command to view the contents of a container.

Directories inside the GridAPPS-D container that you may wish to access:

* `applications`: This directory contains custom applications loaded into the platform

* `log`: This directory contains a copy of the log, which can be viewed by running `cat gridappsd.log`

* `services`: This directory contains available services and service config files

It is also possible to access any other folder in container, such as /tmp by running `cd /tmp` from inside the container

![docker-ls](images/1.6/docker_exec_ls.png)

---

### Exit a Running Container

To exit a running container, simply type `exit` from inside the container

![docker-exit](images/1.6/docker_exit.png)

---

### Kill a Running Container

To stop a running container, use `docker kill container_name`. This command should only be used if a container has crashed or become unresponsive. The GridAPPS-D platform should be stopped with the `./stop.sh` script.

![docker-kill](images/1.6/docker_kill.png)

---

# 3. Managing Container Images

## Managing Container Images

This section lists common docker commands for managing containers and images stored on your hard disk

### Prune Unused Images

To free up hard disk space, delete unused containers and images by running `docker system prune -a`

---

### Delete All Containers

__Use this command with caution! This command will delete *everything*!__

If the docker containers have been corrupted, it may be necessary to delete all containers and images. This will delete all local copies of the GridAPPS-D Platform, all simulation data, and all custom models uploaded to Blazegraph.

* Stop all containers: `docker-compose down`

* Delete all containers: `docker rm -f $(docker ps -a -q)`

* Delete all volumes: `docker volume rm $(docker volume ls -q)`

![docker-delete](images/1.6/docker_delete.png)

---

### Update Containers Manually

Generally, the GridAPPS-D platform automatically checks for new container images when running the `/run.sh` command.

It is also possible to force docker to check for new images and download new containers by running `docker-compose pull`



---

## Transferring Container Data

Files in docker containers are isolated from the rest of the file directories. To edit or run them with another program, such as OpenDSSCMD or GridLAB-D, it is necessary to copy those files using the `docker cp` command

### Transferring Configuration File API Output

The Configuration File API contains two calls for converting the CIM XML power system and [exporting all GridLAB-D files](../api_usage/3.4-Using-the-Configuration-File-API.ipynb#Export-all-GridLab-D-Files) and [exporting all OpenDSS files](../api_usage/3.4-Using-the-Configuration-File-API.ipynb#Export-all-OpenDSS-Files). 

These API calls write the requested files to the directory in the `gridappsd:` container specified by the `"directory": ` key.

To transfer the requested files from inside the container to the machine hard disk, use the `docker cp source_path dest_path` command:

* Default GLM path: `docker cp gridappsd:/tmp/gridlabdsimulation /home/your_username/your_dest_path`

* Default DSS path: `docker cp gridappsd:/tmp/dsssimulation /home/your_username/your_dest_path`



---

### Transferring Simulation GLM Files

During simulation startup, the GridAPPS-D Platform stores the GridLab-D simulation startup files and model in a temporary folder named after the simulation ID. 

When debugging why the simulation solution of a new power system model does not converge, it is extemely useful to copy the GLM solution files to debug the model outside of the docker container. 

First, start a simulation of the desired power system model [using the GridAPPS-D Viz](../installation/1.5-Using-GridAPPS-D-Viz.ipynb#Creating-a-Simulation) or [using the Simulation API](../api_usage/3.5-Creating-Running-Simulation-API.ipynb#Starting-the-Simulation). 

Copy the simulation id from Viz by left-clicking on the simulation ID or from your application's API call:

![copy-sim-id](images/1.6/copy_sim_id.png)

The simulation files can be copied out of the Docker container by running

`docker cp gridappsd:/tmp/gridappsd_tmp/sim_id /home/your_username/your_dest_path`

__Example:__ For the simulation above with simulation ID 78890427, the command to copy the files to a new folder named "my_glm_files" would be 

`docker cp gridappsd:/tmp/gridappsd_tmp/78890427 /home/ubuntu/my_glm_files`

---

## Configuring GridAPPS-D Containers

The GridAPPS-D Platform can be configured and customized by editing the `docker-compose.yml` file in the `gridappsd-docker` directory using your preferred text editor

### Adding Applications 

.
.
.
.

---

### Adding New Services

To add a new underlying service, add the local path for the new service to the `volumes:` section of the gridappsd container:

```
volumes:
    - ~/my_path/my_service:/gridappsd/services/my-service
    - ~/my_path/my_service/my-service.config:/gridappsd/services/my-service.config
```

The example below shows how a repository containing two new services is added manually to the gridappsd-container

![volumes-add-service](images/1.6/volumes_add_service.png)

---

### Changing Tags of a Container

The GridAPPS-D platform can be pinned to use a particular container by specifying the particular release in the `image:` section of the particular container:

* `image: gridappsd/container_name:releases_release_name`

It is not recommended to pin to a specific container except for debugging or if the application uses a deprecated feature from an old GridAPPS-D Platform release.

The example below shows how the main gridappsd container and viz can be pinned to particular releases. 

![image-pinned](images/1.6/image_pinned.png)

---

### Building a Local GridAPPS-D Container

To build and test a local version of the GridAPPS-D container (such as for a feature pull request), first clone the desired branch of the GOSS-GridAPPS-D repository:

* `git clone https://github.com/GRIDAPPSD/GOSS-GridAPPS-D.git -b branch_to_test`


Change directories into GOSS-GridAPPSD folder and run the container build script. It make take a few minutes for the container to build.

* `cd GOSS-GridAPPS-D`

* `./build-gridappsd-container`



![build-container](images/1.6/build_local_container.png)


Then edit the `docker-compose.yml` file and change the gridappsd image to `local`:

* `image: gridappsd:local`

![image-local](images/1.6/image_local.png)

After saving the `docker-compose.yml` file, restart the gridappsd platform by running `./stop.sh` and the `./run.sh`

---

![gridappsd-logo](../images/GridAPPS-D_narrow.png)